In [25]:
from os import environ
from iexfinance.stocks import Stock
from dotenv import load_dotenv
from datetime import datetime
from iexfinance.stocks import Stock
import pyodbc
import sqlalchemy
import pandas
import urllib
from sqlalchemy import create_engine

# splits a list into chunks of the given size
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

# load the environment variables and verity we have a token
load_dotenv()
iex_token = [environ[key] for key in environ if key == 'IEX_TOKEN']
print(iex_token[0][0:10])

Tsk_5c3a24


In [9]:
# connect to the database
drivers = [x for x in pyodbc.drivers() if x.endswith(' for SQL Server')]
print(drivers)
(svr, db, uid, pwd) = environ['MSSQL_SERVER'], environ['MSSQL_DB'], environ['MSSQL_UID'], environ['MSSQL_PWD']
conn = pyodbc.connect(f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={svr};DATABASE={db};UID={uid};PWD={pwd}')

['ODBC Driver 17 for SQL Server']


In [6]:
# get the symbols
sql = "Exec usp_Equity_Read"
symbols = pandas.read_sql(sql,conn)
symbols['Symbol'] = symbols['Symbol'].str.upper()
symbol_list = symbols['Symbol'].tolist()
symbols.head()

,EquityID,Symbol,LastProcessDate
0,3,AAPL,2020-12-12 15:50:12.580
1,13,ADBE,2020-12-15 22:20:09.750
2,32,AMD,2020-12-15 20:30:11.150
3,34,AMGN,2020-12-15 07:40:10.107
4,125,CGNX,2020-12-15 04:20:10.360


In [11]:
# get the trade date id
# today = datetime.today().strftime('%Y-%m-%d')
today = '2020-12-16'
sql = f"SELECT TradedateID FROM dbo.TradeDate WHERE TradeDate = '{today}'"
cursor = conn.cursor()
cursor.execute(sql)
tradeDateID = cursor.fetchone()[0]

In [20]:
# pull the data from iex and add it to the data frame
# iex can only get 100 records at a time
chunks = chunker(symbol_list, 100)
data = symbols[['EquityID', 'Symbol']]
quotes = []
for chunk in chunks:
    batch = Stock(chunk, output_format="pandas")
    quote = batch.get_quote()
    quote = quote[['symbol', 'open', 'close', 'high', 'low', 'volume']]
    quotes.append(quote)
quotes = pandas.concat(quotes)
data = data.merge(quotes, left_on='Symbol', right_on='symbol', how='outer')
data.head()

,EquityID,Symbol,symbol,open,close,high,low,volume
0,3,AAPL,AAPL,132.01,129.23,129.23,128.14,100049844
1,13,ADBE,ADBE,491,501.6,503.4,494.21,2604794
2,32,AMD,AMD,101.02,100.44,101.13,96.69,35042294
3,34,AMGN,AMGN,238.3,237.22,240.88,235.59,2630386
4,125,CGNX,CGNX,79.13,79.04,81.9,77.74,707743


In [21]:
# make the columns match the database table
data.rename(columns = {
    'open': 'Open', 
    'close': 'Close', 
    'high': 'High',
    'low': 'Low',
    'volume': 'Volume'
}, inplace = True) 
data.drop(columns=['symbol'], inplace = True)
data['TradeDate'] = today
data['TradeDateID'] = tradeDateID
data['LastModifiedDate'] = datetime.now()
data.head()

,EquityID,Symbol,Open,Close,High,Low,Volume,TradeDate,TradeDateID,LastModifiedDate
0,3,AAPL,132.01,129.23,129.23,128.14,100049844,2020-12-16,13515,2020-12-17 00:13:24.648518
1,13,ADBE,491,501.6,503.4,494.21,2604794,2020-12-16,13515,2020-12-17 00:13:24.648518
2,32,AMD,101.02,100.44,101.13,96.69,35042294,2020-12-16,13515,2020-12-17 00:13:24.648518
3,34,AMGN,238.3,237.22,240.88,235.59,2630386,2020-12-16,13515,2020-12-17 00:13:24.648518
4,125,CGNX,79.13,79.04,81.9,77.74,707743,2020-12-16,13515,2020-12-17 00:13:24.648518


In [28]:
# write to the database
(svr, db, uid, pwd) = environ['MSSQL_SERVER'], environ['MSSQL_DB'], environ['MSSQL_UID'], environ['MSSQL_PWD']
conn = pyodbc.connect(f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={svr};DATABASE={db};UID={uid};PWD={pwd}')

quoted = urllib.parse.quote_plus(f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={svr};DATABASE={db};UID={uid};PWD={pwd}')
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={quoted}')

data.to_sql('Stage_FactEODData', schema='dbo', con=engine, if_exists='append', index=False)

['ODBC Driver 17 for SQL Server']
